# Práctica 3 - Inteligencia Artificial

### Grado Ingeniería Informática Tecnologías Informáticas - Curso 2019-20

### Técnicas metaheurísticas para optimización
### Búsqueda en espacios de estados

José Luis Ruíz Reina

En esta práctica aplicaremos los algoritmos de búsqueda vistos en clase, viendo cómo se comportan con el problema del 8 puzzle. La práctica tiene tres partes bien diferenciadas:

* __Parte I:__ Representación de problemas de espacios de estados. Veremos una técnica general para hacerlo, y en particular se implementará el problema del ocho puzzle.

* __Parte II:__ Experimentación con los algoritmos implementados. Ejecución de los algoritmos implementados, para la búsuqeda de soluciones a instancias concretas de los problemas.

* __Parte III:__ Calcularemos algunos estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. Así, se comprobarán experimentalmente algunas propiedades de los algoritmos.

El código que se usa en esta práctica está basado principalmente en el código Python que se proporciona con el libro "Artificial Intelligence: A Modern Approach" de S. Russell y P. Norvig (http://code.google.com/p/aima-python, módulo search.py). Las modificaciones al código y la traducción han realizadas por José Luis Ruiz Reina (Depto. de Ciencias de la Computación e Inteligencia Artificial de la Universidad de Sevilla).

##  PARTE I. REPRESENTACIÓN DE ESPACIOS DE ESTADOS

Recuérdese que según lo que se ha visto en clase, la implementación de la representación de un problema de espacio de estados consiste en:

* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
   coste_de_aplicar_accion, si el problema tiene coste.

La siguiente clase Problema representa este esquema general de cualquier problema de espacio de estados. Un problema concreto será una subclase de Problema, y requerirá implementar acciones, aplica y eventualmente __init__, es_estado_final y  coste_de_aplicar_accion. 

In [3]:
class Problema(object):
    """Clase abstracta para un problema de espacio de estados. Los problemas
    concretos habría que definirlos como subclases de Problema, implementando
    acciones, aplica y eventualmente __init__, es_estado_final y
    coste_de_aplicar_accion. Una vez hecho esto, se han de crear instancias de
    dicha subclase, que serán la entrada a los distintos algoritmos de
    resolución mediante búsqueda."""  


    def __init__(self, estado_inicial, estado_final=None):
        """El constructor de la clase especifica el estado inicial y
        puede que un estado_final, si es que es único. Las subclases podrían
        añadir otros argumentos"""
        
        self.estado_inicial = estado_inicial
        self.estado_final = estado_final

    def acciones(self, estado):
        """Devuelve las acciones aplicables a un estado dado. Lo normal es
        que aquí se devuelva una lista, pero si hay muchas se podría devolver
        un iterador, ya que sería más eficiente."""
        abstract

    def aplica(self, estado, accion):
        """ Devuelve el estado resultante de aplicar accion a estado. Se
        supone que accion es aplicable a estado (es decir, debe ser una de las
        acciones de self.acciones(estado)."""
        abstract

    def es_estado_final(self, estado):
        """Devuelve True cuando estado es final. Por defecto, compara con el
        estado final, si éste se hubiera especificado al constructor. Si se da
        el caso de que no hubiera un único estado final, o se definiera
        mediante otro tipo de comprobación, habría que redefinir este método
        en la subclase.""" 
        return estado == self.estado_final

    def coste_de_aplicar_accion(self, estado, accion):
        """Devuelve el coste de aplicar accion a estado. Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Lo que sigue es un ejemplo de cómo definir un problema como subclase de problema. En concreto, el problema de las jarras, visto en clase:

In [4]:
class Jarras(Problema):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        super().__init__((0,0))

    def acciones(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def aplica(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def es_estado_final(self,estado):
        return estado[0]==2

Veamos algunos ejemplos de cómo se usa

In [5]:
pj = Jarras()

In [6]:
pj.estado_inicial
# Resultado: (0, 0)

(0, 0)

In [7]:
pj.acciones(pj.estado_inicial)
# Resultado: ['llenar jarra de 4', 'llenar jarra de 3']

['llenar jarra de 4', 'llenar jarra de 3']

In [8]:
pj.aplica(pj.estado_inicial,"llenar jarra de 4")
# Resultado: (4, 0)

(4, 0)

In [9]:
pj.coste_de_aplicar_accion(pj.estado_inicial,"llenar jarra de 4")
# Resultado: 1

1

In [10]:
pj.es_estado_final(pj.estado_inicial)
# Resultado:False

False

### Ejercicio 1

Definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. Para ello, completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

Pequeña nota sobre el código, hay que usar los strings que vienen marcados en el enunciado a la hora de hacer la función de acción.
Si no los usamos vamos a tener que cambiar los strings en los ejemplos uno a uno o dará error.

In [19]:
class Ocho_Puzzle(Problema):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Por ejemplo, el
    estado final será la tupla (1, 2, 3, 8, 0, 4, 7, 6, 5). Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente. 
    """
    
    """tablero_inicial es la lista con la posición inicial tal que en la teoría vimos que sería (2,1,7,8,6,0,3,4,5)
        así como el estado_final es el que quiere que representemos y que está enunciado en el comentario de encima"""
    
    def __init__(self,tablero_inicial):
        super().__init__(estado_inicial=tablero_inicial, estado_final=(1,2,3,8,0,4,7,6,5))

    def acciones(self,estado):
        # Vemos en qué posición del 8-puzzle está el valor 0 que es el hueco
        pos_hueco=estado.index(0)
        accs=list()
        """Recordamos que en la abstracción del 8-puzzle no deja de ser una lista de tamaño 9 tal que sus 
            índices de POSICIÓN son [0,1,2,3,4,5,6,7,8]
            Los cuáles al estar subdivididos en filas de 3 nos deja un cuadrado en el que tenemos las filas (horizontalidad)
            
            Arriba: [0,1,2]
            Medio: [3,4,5]
            Abajo: [6,7,8]
            
            Y con sus columnas (en el sentido vertical):
            
            Izquierda: [0,3,6]
            Medio: [1,4,7]
            Derecha: [2,5,8]
            
            También cabe destacar que las acciones que hará el hueco en el 8-puzzle será moviéndose en Arriba, Abajo, Derecha e Izquierda
            """
        
        # Si el hueco no está en la posición de arriba ...
        if pos_hueco not in [0,1,2]: 
            # ... se puede mover hacia arriba ...
            accs.append("Mover hueco arriba")
        # Y así con las demás posiciones...
        if pos_hueco not in [6,7,8]: 
            accs.append("Mover hueco abajo")
        if pos_hueco not in [0,3,6]: 
            accs.append("Mover hueco izquierda")
        if pos_hueco not in [2,5,8]: 
            accs.append("Mover hueco derecha")
        return accs     

    def aplica(self,estado,accion):
        pos_hueco = estado.index(0)
        """Vamos a hacer una copia del estado inicial en forma de lista para modificar las posiciones
        sin modificar la tupla del estado inicial. 
        El resultado al final, lo devolveremos en forma de tupla que es lo que nos piden como resultado"""
        resultado = list(estado)
        if accion == "Mover hueco arriba":
            nueva_pos_hueco = pos_hueco - 3
        elif accion == "Mover hueco abajo":
            nueva_pos_hueco = pos_hueco + 3
        elif accion == "Mover hueco izquierda":
            nueva_pos_hueco = pos_hueco - 1
        # if accion == "Mueve el hueco a la derecha":
        else: 
            nueva_pos_hueco = pos_hueco + 1
        # Desplazamos el hueco a la posición nueva según la acción tomada y el valor que estaba en esa posición pasará a estar dónde estaba el hueco
        resultado[pos_hueco], resultado[nueva_pos_hueco] = resultado[nueva_pos_hueco], resultado[pos_hueco]
        # Devolvemos el resultado transformado en tupla
        return tuple(resultado)
        

Ejemplos que se pueden ejecutar una vez se ha definido la clase:

In [20]:
p8p_1 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))

In [21]:
p8p_1.estado_inicial
# Resultado: (2, 8, 3, 1, 6, 4, 7, 0, 5)

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [22]:
p8p_1.estado_final
# Resultado: (1, 2, 3, 8, 0, 4, 7, 6, 5)

(1, 2, 3, 8, 0, 4, 7, 6, 5)

In [23]:
p8p_1.acciones(p8p_1.estado_inicial)
# Resultado: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [24]:
p8p_1.aplica(p8p_1.estado_inicial,"Mover hueco arriba")
# Resultado: (2, 8, 3, 1, 0, 4, 7, 6, 5)

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [25]:
p8p_1.coste_de_aplicar_accion(p8p_1.estado_inicial,"Mover hueco arriba")
# Resultado: 1

1

##  PARTE I. EXPERIMENTANDO

Los algoritmos de búsquedas están implementados el el fichero *algoritmos_de_búsqueda.py*. Importamos las funcionesdefinidas en el módulo.

Vamos a usar los algoritmos que vienen dentro del archivo bajado algoritmos_de_busqueda.py

In [26]:
from algoritmos_de_búsqueda import *

### Ejercicio 2

Usar búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras como al problema del ocho puzzle con distintos estados iniciales. Puedes probar los siguientes ejemplos.

In [27]:
búsqueda_en_anchura(Jarras()).solucion()
# Resultado:
# ['llenar jarra de 4', 'trasvasar de jarra de 4 a jarra de 3', 
#  'vaciar jarra de 3', 'trasvasar de jarra de 4 a jarra de 3', 
#  'llenar jarra de 4', 'trasvasar de jarra de 4 a jarra de 3']

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [28]:
búsqueda_en_profundidad(Jarras()).solucion()
# Resultado:
# ['llenar jarra de 3', 'trasvasar de jarra de 3 a jarra de 4', 
#  'llenar jarra de 3', 'trasvasar de jarra de 3 a jarra de 4', 
#  'vaciar jarra de 4', 'trasvasar de jarra de 3 a jarra de 4']

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

In [29]:
búsqueda_en_anchura(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solucion()
# Resultado:
# ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 
#  'Mover hueco abajo', 'Mover hueco derecha']

['Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [30]:
búsqueda_en_profundidad(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solucion()
# Resultado:
# ['Mover hueco derecha', 'Mover hueco arriba', ... ] # ¡más de 3000 acciones!

['Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover h

### Ejercicio 3

Definir las dos funciones heurísticas para el 8 puzzle que se han visto en clase. Es decir:
* h1_ocho_puzzle(estado): cuenta el número de casillas mal colocadas respecto del estado final.
* h2_ocho_puzzle_estado(estado): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final. 


zip(tupla1, tupla2) es una función para tuplas en la que se juntan las tuplas que están dentro de la función unificando la tupla en una tupla de listas de valores del tipo ((x1,y1),(x2,y2),...(xn,yn)). Referencia: https://www.w3schools.com/python/ref_func_zip.asp

Cuando colocamos el par x,y en un for teniendo zip(tupla1, tupla2) el tamaño de las tuplas van a ser el límite del for además x e y serán cada valor de la tupla por orden respectivo.

In [32]:
# Solución:
def h1_ocho_puzzle(estado):
    casillas_mal_colocadas = 0
    # El estado final
    estado_final = (1,2,3,8,0,4,7,6,5)
    for x,y in zip(estado, estado_final):
        # Si el valor del estado no es la casilla hueca y el valor NO es el mismo con el resultado
        if x!=0 and x!=y:
            # La casilla está mal colocada
            casillas_mal_colocadas += 1
    return casillas_mal_colocadas

La distancia manhattan o distancia euclídea se define como la suma absoluta de la diferencia de los valores entre dos puntos de los ejes definidos.
Referencia: https://eloviparo.wordpress.com/2018/03/13/la-distancia-manhattan-o-la-distancia-euclidea/

Para ello, pondremos como límite el cuadrado del 8-puzzle, que es un 3x3 y por ello los valores que cojamos los dividiremos entre 3.

In [36]:
def h2_ocho_puzzle(estado):
    estado_final = (1,2,3,8,0,4,7,6,5)
    distancia = 0
    # range(9) porque en el 8-puzzle hay 9 casillas
    for i in range(9):
        estado_de_i = estado[i]
        
        # Vamos a hacer el cálculo de la distancia euclídea con aquellos estados que no son el hueco
        if estado != 0:
            # Usamos como j el estado_final como en la anterior función
            j = estado_final.index(estado_de_i)
            
            # ix e iy van a ser el mismo valor usando el módulo 3 anteriormente descrito usando divmod(...)
            ix, iy = divmod(i,3)
            jx, jy = divmod(j,3)
            
            # abs() saca el valor absoluto de la operación que haya dentro
            distancia += abs(ix-jx) + abs(iy-jy)
    return distancia

Lo probamos

In [37]:
h1_ocho_puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
# Resulatado: 4

4

In [38]:
h2_ocho_puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
# Resultado: 5

6

In [39]:
h1_ocho_puzzle((5,2,3,0,4,8,7,6,1))
# Resultado: 4

4

In [40]:
h2_ocho_puzzle((5,2,3,0,4,8,7,6,1))
# Resultado: 11

12

### Ejercicio 4

Resolver usando búsqueda_óptima, búsqueda_primero_el_mejor y búsqueda_a_estrella (con las dos heurísticas), el problema del 8 puzzle parael siguiente estado inicial:

In [ ]:
# Estado inicial

#              +---+---+---+
#              | 2 | 8 | 3 |
#              +---+---+---+
#              | 1 | 6 | 4 |
#              +---+---+---+
#              | 7 | H | 5 |
#              +---+---+---+

In [42]:
búsqueda_óptima(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solucion()
# Resultado:
# ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 
#  'Mover hueco abajo', 'Mover hueco derecha']

['Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [43]:
búsqueda_primero_el_mejor(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5)),h1_ocho_puzzle).solucion()
# Resultado:
# ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
#  'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 
#  'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [44]:
búsqueda_primero_el_mejor(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5)),h2_ocho_puzzle).solucion()
# Resultado:
# ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 
#  'Mover hueco abajo', 'Mover hueco derecha']

['Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [45]:
búsqueda_a_estrella(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5)),h1_ocho_puzzle).solucion()
# Resultado:
# ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 
#  'Mover hueco abajo', 'Mover hueco derecha']

['Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [46]:
búsqueda_a_estrella(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5)),h2_ocho_puzzle).solucion()
# Resultado:
# ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 
#  'Mover hueco abajo', 'Mover hueco derecha']

['Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha']

## PARTE III. Estadísticas

La siguientes definiciones nos van a permitir experimentar con distintos estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. Además se van a contar el número de nodos analizados durante la búsqueda:

In [41]:
class Problema_con_Analizados(Problema):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora un atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algorimo de búsqueda.""" 
         
    def __init__(self, problema):
        self.estado_inicial = problema.estado_inicial
        self.problema = problema
        self.analizados  = 0

    def acciones(self, estado):
        return self.problema.acciones(estado)

    def aplica(self, estado, accion):
        return self.problema.aplica(estado, accion)

    def es_estado_final(self, estado):
        self.analizados += 1
        return self.problema.es_estado_final(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problema.coste_de_aplicar_accion(estado,accion)



def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:

       >>> resuelve_ocho_puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5),búsqueda_a_estrella,h2_ocho_puzzle)
       Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 
                  'Mover hueco abajo', 'Mover hueco derecha']
       Algoritmo: búsqueda_a_estrella
       Heurística: h2_ocho_puzzle
       Longitud de la solución: 5. Nodos analizados: 7
       """

    p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    sol= (algoritmo(p8p,h).solucion() if h else algoritmo(p8p).solucion()) 
    print("Solución: {0}".format(sol))
    print("Algoritmo: {0}".format(algoritmo.__name__))
    if h: 
        print("Heurística: {0}".format(h.__name__))
    else:
        pass
    print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))

### Ejercicio 5

Intentar resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

In [47]:
#           E1              E2              E3              E4
#           
#     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
#     | 2 | 8 | 3 |   | 4 | 8 | 1 |   | 2 | 1 | 6 |   | 5 | 2 | 3 |
#     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
#     | 1 | 6 | 4 |   | 3 | H | 2 |   | 4 | H | 8 |   | H | 4 | 8 |
#     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
#     | 7 | H | 5 |   | 7 | 6 | 5 |   | 7 | 5 | 3 |   | 7 | 6 | 1 |
#     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+  

# Donde va H va el hueco y por tanto el 0
E1 = (2,8,3,1,6,4,7,0,5)
E2 = (4,8,1,3,0,2,7,6,5)
E3 = (2,1,6,4,0,8,7,5,3)
E4 = (5,2,3,0,4,8,7,6,1)

Se pide, en cada caso, hacerlo con la función resuelve_ocho_puzzle, para obtener, además de la solución, la longitud (el coste) de la solución obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla (L, longitud de la solución, NA, nodos analizados), y justificarlos con las distintas propiedades teóricas estudiadas.

In [50]:
# -----------------------------------------------------------------------------------------
#                                       E1           E2           E3          E4
                                
# Anchura                             L=            L=           L=          L=  
#                                     NA=           NA=          NA=         NA= 
resuelve_ocho_puzzle(E1, búsqueda_en_anchura)                                                                              
# Profundidad                         L=            L=           L=          L=  
#                                     NA=           NA=          NA=         NA= 
resuelve_ocho_puzzle(E1, búsqueda_en_profundidad)  
# Óptima                              L=            L=           L=          L=  
#                                     NA=           NA=          NA=         NA= 
resuelve_ocho_puzzle(E1, búsqueda_óptima)                                                                               
# Primero el mejor (h1)               L=            L=           L=          L=
#                                     NA=           NA=          NA=         NA=
resuelve_ocho_puzzle(E1, búsqueda_primero_el_mejor, h1_ocho_puzzle)                                                                           
# Primero el mejor (h2)               L=            L=           L=          L= 
#                                     NA=           NA=          NA=         NA=
resuelve_ocho_puzzle(E1, búsqueda_primero_el_mejor, h2_ocho_puzzle)                                                                             
# A* (h1)                             L=            L=           L=          L= 
#                                     NA=           NA=          NA=         NA=
resuelve_ocho_puzzle(E1, búsqueda_a_estrella, h1_ocho_puzzle)                                                                          
# A* (h2)                             L=            L=           L=          L= 
#                                     NA=           NA=          NA=         NA=
resuelve_ocho_puzzle(E1, búsqueda_a_estrella, h2_ocho_puzzle) 
# -----------------------------------------------------------------------------------------

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: búsqueda_en_anchura
Longitud de la solución: 5. Nodos analizados: 35
Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquier

No vamos a seguir haciendo los demás estados porque lo único que hay que cambiar es la variable de estado que pasa como entrada

### Ejercicio 6

La siguiente heurística h3_ocho_puzzle se obtiene sumando a la heurística h2_ocho_puzzle una componente que cuantifica la "secuencialidad" en las casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en A*, con cada uno de los estados anteriores. Comentar los resultados.

In [51]:
def h3_ocho_puzzle(estado):

    suc_ocho_puzzle ={0: 1, 1: 2, 2: 5, 3: 0, 4: 4, 5: 8, 6: 3, 7: 6, 8: 7}  

    def secuencialidad_aux(estado,i):
        
        val=estado[i]
        if val == 0:
            return 0
        elif i == 4:
            return 1
        else:
            i_sig=suc_ocho_puzzle[i]
            val_sig = (val+1 if val<8 else 1)
            return 0 if val_sig == estado[i_sig] else 2 

    def secuencialidad(estado):
        res= 0 
        for i in range(8): 
            res+=secuencialidad_aux(estado,i)
        return res    

    return h2_ocho_puzzle(estado) + 3*secuencialidad(estado)

In [52]:
resuelve_ocho_puzzle(E2, búsqueda_a_estrella, h3_ocho_puzzle)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda']
Algoritmo: búsqueda_a_estrella
Heurística: h3_ocho_puzzle
Longitud de la solución: 12. Nodos analizados: 18


No podemos hablar de heurísticas admisibles usando el algoritmos a* ya que no entra dentro de la definición de uso admisible ya que no entra en coalición con la heurística entrópica, lo que sí podemos decir es que la longitud de la solución no es la más acertada para los nodos analizados ya que para llegar a su solución hay que recorrerlos en su mayoría